In [17]:
using LowLevelFEM, LinearAlgebra

In [18]:
gmsh.initialize()
gmsh.open("cantilever.geo")

Info    : Reading 'cantilever.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.00295709s, CPU 0.001204s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000858512s, CPU 0.000795s)
Info    : Meshin

In [19]:
mat = material("body")
prob = Problem([mat])

Problem("cantilever", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0e-7, 0.1, 1.0)], 1.0, 1836, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [20]:
Φ = π / 6 * 0
λ = 1.1

R = TensorField(prob, "body", [1 0 0; 0 cos(Φ) sin(Φ); 0 -sin(Φ) cos(Φ)])
U = TensorField(prob, "body", [1 0 0; 0 1 0; 0 0 λ])
F = R * U;

In [21]:
Φ2 = π / 6 * 0 + 0.0001
λ2 = 1.1 + 0.000

R2 = TensorField(prob, "body", [1 0 0; 0 cos(Φ2) sin(Φ2); 0 -sin(Φ2) cos(Φ2)])
U2 = TensorField(prob, "body", [1 0 0; 0 1 0; 0 0 λ2])
F2 = R2 * U2;

In [22]:
R0 = nodePositionVector(prob)
r = F * R0
r2 = F2 * R0
showDeformationResults(r, name="r(t)")
showDeformationResults(r2, name="r(t+Δt)")

1

In [23]:
dr = r2 - r
u = dr;
v = dr;

In [24]:
showDoFResults(v, name="v(t)")

2

In [25]:
F⁺ = r ∘ ∇
F2⁺ = r2 ∘ ∇
dF = v ∘ ∇
l = dF * inv(F);

In [26]:
d = (l + l') / 2;
w = (l - l') / 2;

In [27]:
rnd1 = rand(3)
v1 = VectorField(prob, "body", rnd1 / norm(rnd1))
rnd2 = rand(3)
v2 = VectorField(prob, "body", rnd2 / norm(rnd2))
v2 = v1 × v2 / norm(v1 × v2)
v3 = v1 × v2

ex = VectorField(prob, "body", [1, 0, 0])
ey = VectorField(prob, "body", [0, 1, 0])
ez = VectorField(prob, "body", [0, 0, 1])

Q = v1 ∘ ex + v2 ∘ ey + v3 ∘ ez;

I = TensorField(prob, "body", [1 0 0; 0 1 0; 0 0 1]);

In [28]:
E = (F' * F - I) / 2
E2 = (F2' * F2 - I) / 2
e = inv(F') * E * inv(F)
e2 = inv(F2') * E2 * inv(F2)
de = e2 - e

dE = E2 - E
d = inv(F') * dE * inv(F);

In [29]:
showDoFResults(E, name="E")
showDoFResults(e, name="e")
showDoFResults(dE, name="dE/dt")
showDoFResults(de, name="de/dt")
showDoFResults(d, name="d")
#showDoFResults(de - d, name="de/dt - d")

7

In [30]:
showDoFResults(w * u, name="w.u")
showDoFResults(v - w * u, name="v-w.u")

showDoFResults(de - w * e + e * w, name="de-w.e+e.w")

10

In [31]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [32]:
gmsh.finalize()